# To Does:

- Find out how to break down multiple recipients in one cell into multiple rows while keeping the sender 
- Find out how to extract email body
- Find out how to create a dataframe or dictionary with sender/ recipient information (how many emails has each employee send to and received from all other employees)

## Data source:

Kaggle, The Enron Email Dataset: https://www.kaggle.com/wcukierski/enron-email-dataset
based on the May 7, 2015 Version of dataset, as published at https://www.cs.cmu.edu/~./enron/

In [1]:
# libraries for pre-processing
import pandas as pd
import numpy as np
import re
# libraries for network analysis
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [2]:
df = pd.read_csv('../data/emails.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB


In [4]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [5]:
df.message[0]

"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Tim Belden <Tim Belden/Enron@EnronXGate>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nHere is our forecast\n\n "

In [6]:
print(df.message[0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [7]:
df.file[0]

'allen-p/_sent_mail/1.'

In [8]:
print(df.message[268900])

Message-ID: <12445037.1075840220349.JavaMail.evans@thyme>
Date: Thu, 2 Nov 2000 03:40:00 -0800 (PST)
From: cscusack@email.msn.com
To: kenneth.lay@enron.com
Subject: Ken Derr
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: "cscusack" <cscusack@email.msn.com>
X-To: <kenneth.lay@enron.com>
X-cc: 
X-bcc: 
X-Folder: \Kenneth_Lay_Dec2000\Notes Folders\All documents
X-Origin: LAY-K
X-FileName: klay.nsf

Rosalee, just heard back from Mr. Derr's assistant.  Unfortunately, he is 
unable to attend. Please delete his name from the  letter/invitation. Thank 
you. Chris


In [16]:
df_user = get_user(df)
df_user

,file,message,user
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,allen-p
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,allen-p
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,allen-p
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,allen-p
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,allen-p
...,...,...,...
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...,zufferli-j
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...,zufferli-j
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...,zufferli-j
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...,zufferli-j


In [18]:
df_user.value_counts(['user'])

user      
kaminski-v    28465
dasovich-j    28234
kean-s        25351
mann-k        23381
jones-t       19950
              ...  
swerzbin-m      355
sanchez-m       256
south-s         248
slinger-r       132
phanis-s         35
Length: 150, dtype: int64

### Take a random sample for testing purposes in the pre-processing stage

In [9]:
test = df.sample(n=200, random_state=0)

## Define helper functions for pre-processing the data

Note:
Figure out a way to drop rows based on certain folder names in the file column.

To drop:
- calendar
- schedule_crawler
- funny

Maybe to drop:
- press_releases
- contact
- contacts
- private* (including variations such as private_folders)
- *personal* (including variations such as my_personal)
- tasks
- to_do

In [10]:
def get_user(df):
    """
    Input:
        DataFrame
    Function:
        Extracts the user account in the column 'file' of the DataFrame and appends it to a list. Appends this 
        list as column 'user' to the dataframe and returns it.
    Output:
        DataFrame
    """
    file_split = []
    
    for file in df['file']:
        split = file.rsplit('/')
        file_split.append(split[0])
        
    df['user'] = file_split
        
    return df

In [11]:
# helper function to clean up string in column "message"
def clean_text(message, match):
    """
    Takes a string and the separater to be replaced, replaces linebrakes with empty quotation marks and splits 
    the string at those occurences. Returns a string.
    """ 
    text = re.sub(r"\n\t", "", message)
    return re.split(match, message)

In [12]:
def get_date_time(df):
    """
    Input:
        DataFrame
    Function:
        Extracts the date and time from the column 'message_clean' of the DataFrame, stores the information in
        seperate columns and returns the DataFrame.
    Output:
        DataFrame
    """
    df['date'] = df['message_clean'].apply(lambda x: x[1])
    df.loc[:, 'date'] = df.loc[:, 'date'].apply(lambda x: x[6:31])
    df['date'] = pd.to_datetime(df['date'])
    
    return df

In [13]:
# helper function to extract email addresses from string
def extract_email_string(message, substring):
    """
    Takes a large string (i.e. message) and a substring (i.e. indicator for sender or recipient) to find all
    occurences of a defined regular expression and stores them in a list. If substring is not found in message,
    then the list item remains empty. Returns a list.
    """
    result = re.findall('[\w\.-]+@[\w\.-]+\.\w+', message)
    if substring not in message:
        result = ""
    return result

In [14]:
def get_sender_recipients(df):
    """
    Input:
        DataFrame
    Function:
        Extracts the email addresses of sender, recipient, recipient (cc) and recipient (bcc) from the column 
        'message_clean' of the DataFrame, stores the information in seperate columns and returns the DataFrame.
    Output:
        DataFrame
    """
    
    df['sender'] = df['message_clean'].apply(lambda x : x[2])
    df.loc[:, 'sender'] = df.loc[:,'sender'].apply(lambda x : x[6:])
    
    df['recipients'] = df['message_clean'].apply(lambda x : x[3])
    df.loc[:, 'recipients'] = df.loc[:, 'recipients'].apply(lambda x: x[3:])
    
    test['recipients_cc'] = test['message_clean'].apply(lambda x : x[5])
    
    test['recipients_bcc'] = test['message_clean'].apply(lambda x : x[9])
    
    return df

In [15]:
test['sender'] = test['message_clean'].apply(lambda x : x[2])
test.loc[:, 'sender'] = test.loc[:,'sender'].apply(lambda x : x[6:])

KeyError: 'message_clean'

In [ ]:
test['recipient'] = test['message_clean'].apply(lambda x : x[3])
test.loc[:, 'recipient'] = test.loc[:, 'recipient'].apply(lambda x: x[3:])

In [ ]:
test['recipient_cc'] = test['message_clean'].apply(lambda x : x[5])

In [ ]:
test['recipient_bcc'] = test['message_clean'].apply(lambda x : x[9])

In [ ]:
test['test_message'] = test['message_clean'].apply(lambda x : x[3:])

In [ ]:
mails = test['message_clean'].str.extractall(r'(\w+@\w+\.\w+)')

In [ ]:
test['test_message'] = test['test_message'].str.replace(r'\t','')

In [ ]:
test

In [ ]:
df = test['message'].str.extractall(r'To:([\w\.-]+@[\w\.-]+\.\w+)')

In [ ]:
df

In [ ]:
test['message'][510816]

In [ ]:
test.loc[:, 'recipient_cc'] = test.loc[:, 'recipient_cc'].apply(lambda x: extract_email_string(x, 'Cc:'))

In [ ]:
test.loc[:, 'recipient_bcc'] = test.loc[:, 'recipient_bcc'].apply(lambda x: extract_email_string(x, 'Bcc:'))

### Test helper functions for functionality

In [ ]:
# test clean_text function, store results in new column "message_tidy"
test['message_clean'] = test.message.apply(lambda x: clean_text(x, '\n\t'))

In [ ]:
# test get_email_account function
test = get_user(test)

In [ ]:
# test get_date_time function
test = get_date_time(test)
test

In [ ]:
test['message_clean'][510816]

In [ ]:
test['message_clean'][351661]

## Explore data

In [ ]:
# show number of emails per unique email account, number of unique email accounts is shown in summary line
emails_per_acc = test['user'].value_counts()
emails_per_acc

In [ ]:
"""
add a node for each sender and recipient
for address in kaminski_addresses.keys():
    kam_network.add_node(address, size=kaminski_addresses[address])
for recipient in kaminski_recipients.keys():
    kam_network.add_node(recipient, size=kaminski_recipients[recipient])
    
# add an edge for every email send between two employees
for recipient in kaminski_sender.columns:
    kam_network.add_edge(sender, recipient, weight = kaminski_sender[sender][recipient])
    
"""